In [ ]:
from IPython import get_ipython
from IPython.display import display

In [ ]:
!pip install tool

In [ ]:
!pip install librosa numpy scipy pandas matplotlib

In [ ]:
!pip install soundfile

In [ ]:
import librosa
import numpy as np
import pandas as pd
from scipy.signal import spectrogram, butter, lfilter
from scipy.signal.windows import hamming
import matplotlib.pyplot as plt
import os
import soundfile as sf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Data_preprocessed_python
os.getcwd()
os.chdir('/content/drive/My Drive')

In [ ]:
# Define constants
DEFAULT_SAMPLING_RATE = 44100
SAMPLING_RATE = 11025
CUTOFF_FREQUENCY = 5000
SAMPLES_PER_WINDOW = 4096
TIME_RESOLUTION = SAMPLES_PER_WINDOW / SAMPLING_RATE
UPPER_FREQ_LIMIT = 5000  # Set upper frequency limit for plotting spectrograms
LOWER_FREQ_LIMIT = 20     # Set lower frequency limit for plotting spectrograms


In [ ]:
# Butterworth lowpass filter functions
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def lowpass_filter(data, cutoff=CUTOFF_FREQUENCY, fs=DEFAULT_SAMPLING_RATE, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
# Convert MP3 to WAV and stereo to mono
def convert_mp3_to_wav(mp3_file):
    wav_file = mp3_file.replace('.mp3', '.wav')
    audio, sr = librosa.load(mp3_file, sr=DEFAULT_SAMPLING_RATE, mono=True)  # Convert to mono
    # Use soundfile.write instead of librosa.output.write_wav
    sf.write(wav_file, audio, sr)
    print(f"Converted {mp3_file} to {wav_file}")
    return wav_file

In [ ]:
# Load audio files
def load_audio(file_path):
    audio, sr = librosa.load(file_path, sr=DEFAULT_SAMPLING_RATE)
    return audio

In [ ]:
# Generate spectrogram with Hamming window
def generate_spectrogram(audio):
    window = hamming(SAMPLES_PER_WINDOW)
    frequencies, times, Sxx = spectrogram(audio, fs=DEFAULT_SAMPLING_RATE,
                                          window=window,
                                          nperseg=SAMPLES_PER_WINDOW)
    return frequencies, times, Sxx

In [ ]:
# Plot and save spectrograms as images
def plot_and_save_spectrogram(Sxx, freqs, times, title='Spectrogram', file_name='spectrogram.png'):
    plt.figure(figsize=(10, 6))

    # Masking frequencies outside the desired range for plotting
    freq_mask = (freqs >= LOWER_FREQ_LIMIT) & (freqs <= UPPER_FREQ_LIMIT)

    plt.pcolormesh(times, freqs[freq_mask], 10 * np.log10(Sxx[freq_mask]), shading='gouraud')

    plt.title(title)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.colorbar(label='Intensity [dB]')
    plt.savefig(file_name)
    plt.close()

In [ ]:
# Step 9: Prepare dataset of songs and input audio segment
def prepare_dataset(song_files):
    for song in song_files:
        convert_mp3_to_wav(song)

In [ ]:
# Main function to process everything and generate images for training CNN
def main(test_audio_path, reference_audio_path):
    # Convert reference MP3 file to WAV format
    wav_reference_path = convert_mp3_to_wav(reference_audio_path)
    wav_test_path = convert_mp3_to_wav(test_audio_path)

    # Load input audio and converted reference song audio
    input_audio = load_audio(wav_test_path)
    song_audio = load_audio(wav_reference_path)

def main(input_audio_path, test_audio_path):
    # List of song files (replace with actual paths)
    song_files = [os.path.join(audio_dir, f'song_{i}.mp3') for i in range(1, 11)]

    # Create directory for saving spectrogram images if it doesn't exist
    if not os.path.exists('spectrograms'):
        os.makedirs('spectrograms')

    # Process each song and generate spectrograms
    for index in range(len(song_files)):
        wav_song_path = song_files[index].replace('.mp3', '.wav')
        # Load the audio file using librosa
        audio, sr = librosa.load(wav_song_path, sr=DEFAULT_SAMPLING_RATE) # Added sr argument
        frequencies_song,times_song,Sxx_song=generate_spectrogram(audio)

        # Save spectrogram image for each song with name img_i.png
        plot_and_save_spectrogram(Sxx_song,
                                   frequencies_song,
                                   times_song,
                                   title=f'Spectrogram of {song_files[index]}',
                                   file_name=f'spectrograms/img_{index + 1}.png')

    # Prepare dataset by converting MP3 files to WAV format
    prepare_dataset(song_files)
    # Process input audio segment to generate its spectrogram image
    input_audio = load_audio(input_audio_path)
    frequencies_input,times_input,Sxx_input=generate_spectrogram(input_audio)

    # Save spectrogram image for input audio segment
    plot_and_save_spectrogram(Sxx_input,
                     frequencies_input,
                     times_input,
                     title='Spectrogram of Input Audio',
                     file_name='spectrograms/test_audio.png')

# Example usage (replace with actual paths to your files)
input_audio_path='/content/drive/My Drive/BTL_ML_2024/song_1.mp3'     # Path to your input MP3 audio clip (segment)
test_audio_path='/content/drive/My Drive/BTL_ML_2024/test_audio.mp3'
audio_dir = '/content/drive/My Drive/BTL_ML_2024/'
main(input_audio_path, test_audio_path) # Pass both arguments to main

<ipython-input-24-7aa1c0db7f12>:8: RuntimeWarning: divide by zero encountered in log10
  plt.pcolormesh(times, freqs[freq_mask], 10 * np.log10(Sxx[freq_mask]), shading='gouraud')


Converted /content/drive/My Drive/BTL_ML_2024/song_1.mp3 to /content/drive/My Drive/BTL_ML_2024/song_1.wav
Converted /content/drive/My Drive/BTL_ML_2024/song_2.mp3 to /content/drive/My Drive/BTL_ML_2024/song_2.wav
Converted /content/drive/My Drive/BTL_ML_2024/song_3.mp3 to /content/drive/My Drive/BTL_ML_2024/song_3.wav
Converted /content/drive/My Drive/BTL_ML_2024/song_4.mp3 to /content/drive/My Drive/BTL_ML_2024/song_4.wav
Converted /content/drive/My Drive/BTL_ML_2024/song_5.mp3 to /content/drive/My Drive/BTL_ML_2024/song_5.wav
Converted /content/drive/My Drive/BTL_ML_2024/song_6.mp3 to /content/drive/My Drive/BTL_ML_2024/song_6.wav
Converted /content/drive/My Drive/BTL_ML_2024/song_7.mp3 to /content/drive/My Drive/BTL_ML_2024/song_7.wav
Converted /content/drive/My Drive/BTL_ML_2024/song_8.mp3 to /content/drive/My Drive/BTL_ML_2024/song_8.wav
Converted /content/drive/My Drive/BTL_ML_2024/song_9.mp3 to /content/drive/My Drive/BTL_ML_2024/song_9.wav
Converted /content/drive/My Drive/BTL